In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt 
import torch
import torch.nn as nn
import torch.nn.functional as F
from PositionalEncoding import PositionalEncoding

import sys
sys.path.insert(0, '../../DataGeneration/Python/')
sys.path.insert(0, '../')

import Dataloader

import Sample


In [7]:
d_model = 8
dropout = 0
seqlen = 10 
batch_size = 16
d_output = 27

dataset_path = '../../Dataset/CCVal300.npy'
dataset = np.load(dataset_path,allow_pickle=True)

In [3]:
def process_input(dataset):
    """Pulls inputs from dataset and returns an array (dataset_size, 10, 5) normalized across the 1st dimension"""
    y = np.array([*dataset])[:,0]
    y = np.array([np.array(z) for z in y])
    std = np.std(y, axis=-1, keepdims=True)
    mean = np.mean(y, axis=-1, keepdims=True)
    std[std==0] = .0001
    y = (y - mean) / std
    y = np.transpose(y, (0, 2, 1))
    return y


In [16]:
y = np.array([*dataset])[:,0]
y = np.array([np.array(z) for z in y])
std = np.std(y, axis=1, keepdims=True)
mean = np.mean(y, axis=1, keepdims=True)
std[std==0] = .0001
y = (y - mean) / std
y = np.transpose(y, (0, 2, 1))

In [19]:
dataset[0][0]

(array([-0.,  3.,  6.,  8., 10., 12., 14., 16., 18., 20.]),
 [8.276,
  9.388,
  10.486,
  11.178,
  11.846,
  12.514000000000001,
  13.18,
  13.846,
  15.48,
  21.0],
 [7.816,
  9.024000000000001,
  10.162,
  10.898,
  11.586,
  12.264000000000001,
  12.938,
  13.61,
  14.334,
  16.948],
 [0.002,
  8.568,
  9.788,
  10.562,
  11.288,
  11.984,
  12.672,
  13.352,
  14.030000000000001,
  15.432],
 [0.002, 7.908, 9.33, 10.154, 10.94, 11.666, 12.372, 13.066, 13.754, 14.538])

(array([[-1.72648029, -1.30081071, -1.62989114, -2.64081536, -2.61915945],
        [-1.24242039, -0.97527137, -1.15453902, -0.53960371, -0.62252705],
        [-0.7583605 , -0.65383054, -0.70673213, -0.24034172, -0.26340597],
        [-0.4356539 , -0.45124671, -0.41711362, -0.05048206, -0.05530768],
        [-0.11294731, -0.2556889 , -0.14638327,  0.12760335,  0.14319385],
        [ 0.20975929, -0.0601311 ,  0.12041204,  0.29832986,  0.32654259],
        [ 0.53246588,  0.13484121,  0.38563335,  0.467094  ,  0.5048404 ],
        [ 0.85517248,  0.32981351,  0.65006764,  0.63389577,  0.68010765],
        [ 1.17787907,  0.80816898,  0.93496411,  0.80020694,  0.85385962],
        [ 1.50058567,  2.42415563,  1.96358204,  1.14411293,  1.05185605]]),
 array([ 21.  ,   3.  ,   2.87,   2.06,  45.  , 103.8 ,   7.  ,   6.8 ,
          3.9 ,  10.9 ,  10.7 ,   3.5 ,  10.6 ,  14.2 ,   2.8 ,   1.4 ,
          2.79,   3.02,   0.5 ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
          0.  ,   0.  ,   0.  ])

In [8]:
def process_input(dataset):
    """Pulls inputs from dataset and returns an array (dataset_size, 10, 5) normalized across the 1st dimension"""
    y = np.array([*dataset])[:,0]
    y = np.array([np.array(z) for z in y])
    std = np.std(y, axis=-1, keepdims=True)
    mean = np.mean(y, axis=-1, keepdims=True)
    std[std==0] = .0001
    y = (y - mean) / std
    y = np.transpose(y, (0, 2, 1))
    return y
    
def process_output(dataset, d_output: int) -> np.array([]):
    """Input: dataset. Returns the concatenated values of Sample object 0-padded to be size d_output"""
    Y = np.array([*dataset])[:,1]
    ret = np.zeros((Y.shape[0], d_output))
    for i, y in enumerate(Y):
        t = [y.duration, y.num_events, y.basement_cond, y.surface_hp, y.hp_depth, y.surf_hf, y.surf_temp, 
        *y.exhumation_history.flatten(), *y.exhumation_rates]
        t = np.pad(t, (0, d_output-len(t)), constant_values=0) #Pad
        ret[i] = t
    return ret

def formattedLoader(d_output, dataset_path):

In [32]:
t = [np.array([obj.duration, obj.num_events, obj.basement_cond, obj.surface_hp, obj.hp_depth, 
                       obj.surf_hf, obj.surf_temp, *obj.exhumation_history.flatten(), 
                       *obj.exhumation_rates]) for obj in y]

In [56]:
ret.shape

(300, 27)